## Scraping MLS Player Stats

Scraping www.mlssoccer.com/ for player statistics

### Import Necessary Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime 
from dateutil import parser
import os
import time
import itertools
import re
import pickle

### Scrape Statistics

#### Define Scraper and Helpers

In [2]:
def send_request(url, params={}, attempts=3):

    cnt = 0
    while cnt <= attempts:
        cnt += 1        
        # Send Request
        response = requests.get(url, params)
        # Check Status
        print(response.url, '\n', response.status_code, response.reason)
        if (response.status_code >= 200) and (response.status_code < 300):           
            return response
        # If bad status, pause before trying again
        print('Pause, then retry')
        time.sleep(15)
    return None

In [3]:
def map_id_to_club(franchise_id):
    
    franchise_map = {11091: 'ATL',
                     1207: 'CHI',
                     436: 'COL',
                     454: 'CLB',
                     1326: 'DC',
                     1903: 'DAL',
                     1897: 'HOU',
                     1230: 'LA',
                     11690: 'LAFC',
                     6977: 'MNUFC',
                     1616: 'MTL',
                     928: 'NE',
                     9668: 'NYCFC',
                     399: 'NYRB',
                     6900: 'ORL',
                     5513: 'PHI',
                     1581: 'POR',
                     1899: 'RSL',
                     1131: 'SJ',
                     3500: 'SEA',
                     421: 'KC',
                     2077: 'TOR',
                     1708: 'VAN'}
    return franchise_map[franchise_id]
        

In [4]:
def check_for_data(stats_table):
    # Check if the table has any 
    odd_rows = stats_table.findAll('tr', {'class': 'odd'})
    if odd_rows[0].text.strip() == 'Stats Unavailable':
        return False
    return True    

In [5]:
def split_player_name(df):
    first = []
    last = []
    for t in df.Player.str.split():
        if len(t) == 1:
            first.append(t[0])
            last.append('')
        elif len(t) == 2:
            first.append(t[0])
            last.append(t[1])
        else:
            first.append(t[0])
            last.append(' '.join(t[1:]))
    
    df['Last'] = last
    df['First'] = first  
    
    df = df.drop('Player', axis=1)
    
    return df
        

In [6]:
# # Find and parse franchise popup menu to get list of teams
# soup = BeautifulSoup(response.text, 'lxml')
# franchise_select = soup.find('select', {'id': 'edit-franchise', 'name': 'franchise'})
# franchise = []
# # start with second index since first is generic "Select A Club"
# for team in franchise_select.findAll('option')[1:]:
#     franchise.append(int(team['value']))

In [7]:
def generate_param_combos(param_groups):
    params = []
    for combo in itertools.product(*param_groups):
        params.append({'franchise': combo[0],
                       'group': combo[1],
                       'season_type': combo[2],
                       'year': combo[3],
                       'page': combo[4]})
    return params

In [8]:
def scrape_player_stats(base_url, params, return_last_pg=False):

    response = send_request(base_url, params)
    if response is None:
        return None
    
    # Parse HTML
    soup = BeautifulSoup(response.text,'lxml')
    
    stats_table = soup.find('table')
    
    if not check_for_data(stats_table):
        return None
    
    stats_df = extract_stats(stats_table)
    # Add Year Column
    stats_df['Year'] = np.repeat(int(params['year']), len(stats_df))
    # Add Season Column
    stats_df['Season'] = np.repeat(params['season_type'], len(stats_df))
    
    if return_last_pg:
        return stats_df, get_last_page(soup)
    
    return stats_df     

In [9]:
url = 'https://www.mlssoccer.com/stats/season?franchise=select&group=assists&season_type=REG&year=2009&page=14'

In [10]:
response = send_request(url)

https://www.mlssoccer.com/stats/season?franchise=select&group=assists&season_type=REG&year=2009&page=14 
 200 OK


In [11]:
soup = BeautifulSoup(response.text, 'lxml')

In [12]:
stat_table = soup.find('table')
# stat_table.findAll('tr')[-1]

In [13]:
    # Extract salary data
    stat_header = []
    stat_data = []
    t = []
    for row in stat_table.findAll('tr'):
        row_data = []

        # Get row type and check if header or data row
        row_type = row.findChild().name
        if row_type == 'th':
            # Extract header
            for h in stat_table.findAll('th'):
                stat_header.append(h.text) 
        else:
            # Extract data
            for data in row.findAll('td'):
                row_data.append(data.text)
            stat_data.append(row_data) 

In [14]:
def extract_stats(stat_table):
    
    # Extract salary data
    stat_header = []
    stat_data = []
    for row in stat_table.findAll('tr'):
        row_data = []

        # Get row type and check if header or data row
        row_type = row.findChild().name
        if row_type == 'th':
            # Extract header
            for h in stat_table.findAll('th'):
                stat_header.append(h.text) 
        else:
            # Extract data
            for data in row.findAll('td'):
                row_data.append(data.text)
            stat_data.append(row_data)    
            
    # Compile stat dataframe
    stat_df = pd.DataFrame(stat_data, columns=stat_header)
    
    # Strip any whitespace from column names
    stat_df.columns = stat_df.columns.str.strip()
    
    # Replace blanks with nans
    stat_df = stat_df.replace('', np.nan)
        
    return stat_df

In [15]:
def get_last_page(soup):

    last_pg_url = soup.find('li', {'class': 'pager-last last'}).find('a')['href']
    last_pg = int(re.search('(?<=page=)\d+', last_pg_url).group())
    
    return last_pg

In [16]:
def scrape_group_stats(base_url, group, year=np.arange(2007, 2018), franchise='select', season_type='REG'):
    
    group_df = pd.DataFrame()
    combos = generate_param_combos([[franchise], [group], [season_type], year, [0]])
    for params in combos: 
        stats_df, last_pg = scrape_player_stats(base_url, params, True)
        
        if stats_df is None:
            continue
            
        # Scrape first page of results
        df = pd.concat([group_df, stats_df], axis=0)
        # Add pause to prevent 429 status
        print(datetime.now())
        time.sleep(np.random.uniform(2, 5)) 
        
        # Scrape next through last
        for idx in range(1, last_pg+1):
            params['page'] = idx
            stats_df = scrape_player_stats(base_url, params)
            if stats_df is None:
                continue
            group_df = pd.concat([group_df, stats_df], axis=0)            
            # Add pause to prevent 429 status
            print(datetime.now())
            time.sleep(np.random.uniform(2, 5)) 
            
    # Reset index to remove duplicates created during concatenation
    group_df.reset_index(inplace=True, drop=True)
    
    return group_df        

#### Send Request to Base URL and Verify Site is Up

In [17]:
base_url = 'http://www.mlssoccer.com/stats/season'
response = requests.get(base_url)
response.status_code, response.url

(200, 'https://www.mlssoccer.com/stats/season')

#### Define parameter options

In [18]:
# group = ['goals', 'assists', 'shots', 'fouls', 'goalkeeping']
# for grp in group:
#     print('Scraping:', grp, datetime.now(), '\n')
#     df = scrape_group_stats(base_url, grp, np.arange(2007, 2018, 1))
#     df.to_pickle(grp + '_df.pkl')
#     print('Completed Scraping:', grp)

In [19]:
 goalkeeping_df = scrape_group_stats(base_url, 'goalkeeping')

https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2007&page=0 
 200 OK
2018-01-26 12:39:53.737439
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2007&page=1 
 200 OK
2018-01-26 12:40:00.553835
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2008&page=0 
 200 OK
2018-01-26 12:40:07.117394
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2008&page=1 
 200 OK
2018-01-26 12:40:12.595220
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2009&page=0 
 200 OK
2018-01-26 12:40:19.923812
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2009&page=1 
 200 OK
2018-01-26 12:40:26.338721
https://www.mlssoccer.com/stats/season?franchise=select&group=goalkeeping&season_type=REG&year=2010&page=0 
 200 OK
2018-01-26 12:40:32.919909

In [20]:
goalkeeping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 19 columns):
Player    369 non-null object
Club      368 non-null object
POS       369 non-null object
GP        368 non-null object
GS        368 non-null object
MINS      368 non-null object
SHTS      368 non-null object
SV        368 non-null object
GA        368 non-null object
GAA       368 non-null object
PKG/A     369 non-null object
W         368 non-null object
L         368 non-null object
T         368 non-null object
ShO       368 non-null object
W%        369 non-null object
Sv%       369 non-null object
Year      369 non-null int64
Season    369 non-null object
dtypes: int64(1), object(18)
memory usage: 54.9+ KB


In [21]:
goalkeeping_df

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
0,Jay Nolly,DC,GK,1,1,90,9,7,2,2.00,0/0,0,1,0,0,0.0,77.8,2007,REG
1,Steve Cronin,LA,GK,1,1,90,8,6,2,2.00,0/0,0,1,0,0,0.0,75.0,2007,REG
2,David Monsalve,TOR,GK,1,1,90,10,7,3,3.00,0/0,0,1,0,0,0.0,70.0,2007,REG
3,Justin Myers,CHV,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
4,Matt Wideman,RSL,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
5,Lance Friesz,LA,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
6,Kyle Reynish,RSL,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
7,Jordan James,HOU,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
8,Nick Noble,CHI,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG
9,Bill Gaudette,CLB,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2007,REG


In [22]:
goalkeeping_df.to_pickle('goaling_df.pkl')

### Sandbox

In [32]:
goalkeeping_df.convert_objects(convert_numeric=True)

/home/cneiderer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
0,Jay Nolly,DC,GK,1.0,1.0,90.0,9.0,7.0,2.0,2.0,0/0,0.0,1.0,0.0,0.0,0.0,77.8,2007,REG
1,Steve Cronin,LA,GK,1.0,1.0,90.0,8.0,6.0,2.0,2.0,0/0,0.0,1.0,0.0,0.0,0.0,75.0,2007,REG
2,David Monsalve,TOR,GK,1.0,1.0,90.0,10.0,7.0,3.0,3.0,0/0,0.0,1.0,0.0,0.0,0.0,70.0,2007,REG
3,Justin Myers,CHV,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
4,Matt Wideman,RSL,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
5,Lance Friesz,LA,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
6,Kyle Reynish,RSL,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
7,Jordan James,HOU,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
8,Nick Noble,CHI,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG
9,Bill Gaudette,CLB,GK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.0,0.0,0.0,0.0,2007,REG


In [40]:
goalkeeping_df[pd.to_numeric(goalkeeping_df.GP) > 10]

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
319,Andrew Tarbell,SJ,GK,11,11,990,65,40,25,2.27,0/1,4,5,2,2,36.4,61.5,2017,REG


In [43]:
goalkeeping_df.GP.describe()

count     368
unique     12
top         0
freq      224
Name: GP, dtype: object

In [46]:
goalkeeping_df.Player

0                   Jay Nolly
1                Steve Cronin
2              David Monsalve
3                Justin Myers
4                Matt Wideman
5                Lance Friesz
6                Kyle Reynish
7                Jordan James
8                  Nick Noble
9               Bill Gaudette
10              Justin Hughes
11                Shawn Crowe
12              Eric Kronberg
13              Chris Konopka
14                Boris Pardo
15              Brad Knighton
16               Danny Cepero
17           Tomer Chencinski
18                Doug Warren
19            Andy Gruenebaum
20              Jose Carvallo
21     Caleb Patterson-Sewell
22               Lance Parker
23                 Josh Lambo
24              Corbin Waller
25              Charles Alamo
26               Vito Higgins
27              Kenny Schoeni
28             David Monsalve
29              Justin Hughes
                ...          
339            Paolo Tornaghi
340          Earl Edwards Jr.
341       

In [47]:
goalkeeping_df[goalkeeping_df.Club == 'NY']

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
126,Luis Robles,NY,GK,4,4,360,12,9,3,0.75,0/0,2,1,1,2,50.0,75.0,2012,REG
175,Ryan Meara,NY,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2013,REG
225,Ryan Meara,NY,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2014,REG
253,Ryan Meara,NY,GK,1,1,90,7,6,1,1.00,0/0,0,1,0,0,0.0,85.7,2015,REG
298,Kyle Reynish,NY,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2016,REG
300,Ryan Meara,NY,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2016,REG
367,Ryan Meara,NY,GK,0,0,0,0,0,0,0,0/0,0,0,0,0,0.0,0.0,2017,REG
